In [75]:
from pdfminer.high_level import extract_text
import re
import os
from os.path import join
import json
import unicodedata
from PyPDF2 import PdfReader

def pdf_reader(pdf):
    
    reader = PdfReader(pdf)
    pages = [page.extract_text() for page in reader.pages]
    pdf_text = '{PB}'.join(pages)
    
    return(pdf_text)

data_dir = join('..', 'data')
pdf_files = [file for file in os.listdir(data_dir) if file.endswith('.pdf')]

all_files = []

for pdf in pdf_files:
    pdf_dic = {}
    
    file_p = join(data_dir, pdf)
    text_pdfminer = extract_text(file_p)
    text_pypdf = pdf_reader(file_p)
    
    pdf_dic['filename'] = pdf
    pdf_dic['text_pdfminer'] = text_pdfminer
    pdf_dic['text_pypdf'] = text_pypdf
    
    all_files.append(pdf_dic)

In [76]:
#all_files[0].get('text')

In [77]:
n_total = 227 # der bør være 227 afgørelser i alt

In [78]:
def is_pua(c): #https://stackoverflow.com/questions/56337705/removing-all-invalid-characters-e-g-uf0b7-from-text
    return unicodedata.category(c) in ['Co', 'Cc', 'Cf', 'Cs', 'Co', 'Cn']

def text_splitter(text_dict):
    #pattern = re.compile(r'\s{0,5}\n(?=\s{0,4}Ankestyrelsen\s{0,4}\n)', re.DOTALL)
    #pattern = re.compile(r'\s{0,5}\n\s{0,5}A ?n ?k ?e ?s ?t ?y ?r ?e ?l ?s ?e ?n ?\s{0,5}\n\s{0,5}(?!.{0,2}7998)', re.DOTALL)
    pattern = re.compile(r'{PB}(?=.{0,500}A\s?f\s?g\s?ø\s?r\s?e\s?l\s?s\s?e\s?r?\s{0,5}\n\s{0,5}D\s?u\s? \s?h\s?a\s?r\s? \s?k\s?l\s?a\s?g\s?e\s?t\s? \s?o\s?v\s?e\s?r\s?)', re.DOTALL)
    pattern_replace = re.compile(r'(?<=\n)G\'(?=\s)')
    num_sq_replace = re.compile(r'\[\s?\d\s?\]')
    
    text_strip = text_dict.get('text_pypdf')
    text_strip = text_strip.replace('\n', '{LINJESKIFT}')
    text_strip = ''.join(c for c in text_strip if not is_pua(c))
    text_strip = text_strip.replace('{LINJESKIFT}', '\n')
    text_strip = re.sub(pattern_replace, '', text_strip)
    text_strip = re.sub(num_sq_replace, ' ', text_strip)
    text_strip = text_strip.replace('  ', ' ')
    text_strip = text_strip.replace(' -', '-')
    text_strip = text_strip.replace('- ', '-')
    
    
    texts_split = pattern.split(text_strip)
    texts_split.pop(0) # første fundet tekst altid overflødig
    
    texts_return = []
    n = 1
    for text in texts_split:
        if len(text) < 200:
            continue
        
        text_returndict = {'filename': text_dict.get('filename'),
                           'text': text,
                           'n': n,
                           'doc_text': text_dict.get('text_pdfminer')}
        
        texts_return.append(text_returndict)
        
        n = n + 1
    
    return(texts_return)

In [79]:
texts_combined = []

for file in all_files:
    
    texts = text_splitter(file)
    
    texts_combined = texts_combined + texts

In [80]:
print(f'Der mangler {n_total-len(texts_combined)} afgørelser') 

Der mangler 39 afgørelser


In [81]:
out_p = join('..', 'data', 'afgørelser_split.json')

with open(out_p, 'w') as f:
    json.dump(texts_combined, f)